#  Link PyPower

In [1]:
import pandas as pd

import ams
import andes

print(ams.__version__)
print(andes.__version__)

0.5+77.g057140f.dirty
1.8.5


In [2]:
# NOTE: get a pypower case
from ams.solver.ipp import load_ppc

ppc = load_ppc(ams.get_case('pypower/case14.py'))

ppc.keys()

dict_keys(['version', 'baseMVA', 'bus', 'gen', 'branch', 'gencost'])

In [3]:
sa = andes.load(andes.get_case('ieee14/ieee14_ace.xlsx'),
                no_output=True,
                setup=True,)

In [4]:
sp = ams.load(ams.get_case('ieee14/ieee14.xlsx'),
            setup=True)

In [5]:
dct = sp.as_dict()

In [6]:
dct.keys()

odict_keys(['Bus', 'PQ', 'PV', 'Slack', 'Shunt', 'Line', 'Area', 'GCost'])

In [7]:
bus_df = sp.Bus.as_df()
pq_df = sp.PQ.as_df()

In [8]:
bus_df.iloc[0:2]

,idx,u,name,Vn,vmax,vmin,v0,a0,xcoord,ycoord,area,zone,owner
uid,,,,,,,,,,,,,
0,1,1.0,BUS1,69.0,1.1,0.9,1.0300,0.000000,0,0,1,1,1
1,2,1.0,BUS2,69.0,1.1,0.9,1.0197,-0.027981,0,0,1,1,1


In [9]:
pq_df.iloc[0:2]

,idx,u,name,bus,Vn,p0,q0,vmax,vmin,owner
uid,,,,,,,,,,
0,PQ_1,1.0,PQ_1,2,69.0,0.217,0.127,1.2,0.8,1
1,PQ_2,1.0,PQ_2,3,69.0,0.500,0.250,1.2,0.8,1


In [10]:
from ams.solver.ipp import to_ppc

In [11]:
spc = to_ppc(sp)

In [12]:
spc['bus'][:, [4, 5]]

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0., 19.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0., 15.]])

In [13]:
ppc['bus'][:, [4, 5]]

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0., 19.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]])